# Process the data obtained from polarimetry and reorganize the data into the correct folders

In [5]:
%load_ext autoreload
%autoreload 2

import os
from processingmm import batch_processing

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
# set the parameters to run the script
directories = ['/media/elea/ssd/bernDB']
calib_directory = '/media/elea/ssd/bernDB/calib'
    
# set the parameters to be used for the line visualisation
# NB: parameter file accessible in ./src/processingmm/data/parameters_visualisations.json
parameter_set = 'default'

# set run_all to true in order to run the pipeline on all the folders (even the ones already processed)
run_all = False

# PDDN_mode can be set to:
# 1. 'no': processes without using the PDDN
# 2. 'pddn': processes with PDDN when available (for 550nm and 650nm)
# 3. 'both': processes both with PDDN when available and without PDDN
PDDN_mode = 'no'

# Set the wavelengths to be processed
# 1. 'all': processes all the available wavelenght
# 2. [xxx, yyy]: processes only the wavelenghts 'xxx' and 'yyy'
wavelengths = [550]

# Processing mode
# 1. 'no_viz': processes only the MM - no visualization at all. useful for fast computation
# 2. 'default': processes the MM and plots the polarimetric parameters maps (i.e. depolarization, azimuth, 
# retardance, diattenuation, azimuth local variability)
# 3. 'full': do like default, and additionally plot the MM components, as well as the line
# visualization

# define if pdf figures should be saved (takes a lot of time) - no impact when processing_mode is set to no_viz

# NB: processing time without PDDN
# 'no_viz': 0.71s
# 'default', save_pdf_figs False: 2.25s
# 'default', save_pdf_figs True: 3.60s
# 'full', save_pdf_figs False: 3.95s
# 'full', save_pdf_figs True: 8.07s
processing_mode = 'no_viz'
save_pdf_figs = True


parameters = batch_processing.get_parameters(directories, calib_directory, wavelengths, parameter_set = parameter_set, 
                                PDDN_mode = PDDN_mode, processing_mode = processing_mode, run_all = run_all, 
                                save_pdf_figs = save_pdf_figs)

In [7]:
%%time
times = batch_processing.batch_process_master(parameters)
times

processing without PDDN...


0it [00:00, ?it/s]

processing without PDDN done.
CPU times: user 21.4 ms, sys: 55.8 ms, total: 77.1 ms
Wall time: 76.4 ms


{'total': 0}

In [24]:
%%time

# Approximate time to process the line visualization save_pdf_figs True : 3.85s
# Approximate time to process the line visualization save_pdf_figs False : 4.92s
times = batch_processing.batch_visualization(parameters)
times

CPU times: user 5.45 s, sys: 434 ms, total: 5.89 s
Wall time: 5.29 s


5.287709474563599

In [26]:
# align the measurements captured at different wavelengths - could cause issue when using masks obtained
# at 550nm as the images are slightly shifted 
from processingmm.addons import align_wavelengths
PDDN_mode = 'both'
run_all = False
align_wavelengths.align_wavelenghts(directories, PDDN_mode, run_all)

Aligning wavelenght: 600nm


100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 5555.37it/s]


Aligning wavelenght: 650nm


  0%|                                                     | 0/1 [00:00<?, ?it/s]

 
 >> Exported X3D .cod file as: /media/elea/ssd/test/2022-07-06_T_AUTOPSY-BF_FR_S8_1/raw_data/650nm/650_Intensite_aligned.cod


100%|█████████████████████████████████████████████| 1/1 [00:08<00:00,  8.44s/it]


In [39]:
# add the calibration matrices (A.cod and W.cod) to the folder for processing
# needed by Christopher
from processingmm.addons import add_calibration
add_calibration.add_calibration(directories)

100%|█████████████████████████████████████████████| 2/2 [00:00<00:00, 15.59it/s]

No calibration files in 600nm
